In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64

from animal_shelter import AnimalShelter

# MongoDB Connection
username = "aacuser"
password = "Justice01!"
shelter = AnimalShelter(username, password)

# Load all data initially
df = pd.DataFrame.from_records(shelter.read_all({}))
df.drop(columns=['_id'], inplace=True)

# Initialize the Dash app
app = JupyterDash(__name__)

# Grazioso Salvare's logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard Layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Hr(),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
        html.B(html.H3('Developed by Darius Quick')),
    ]),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'Reset'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'}
            ],
            value='Reset',
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        filter_action='native',
        sort_action='native',
        style_table={'height': '300px', 'overflowY': 'auto'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ]),
])

# Callbacks

# Update DataTable based on filter selection
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = {}
    
    if filter_type == 'Water':
        query = {'$and': [
            {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}},
            {'sex_upon_outcome': 'Intact Female'},
            {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
        ]}
    elif filter_type == 'Mountain':
        query = {'$and': [
            {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}},
            {'sex_upon_outcome': 'Intact Male'},
            {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
        ]}
    elif filter_type == 'Disaster':
        query = {'$and': [
            {'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}},
            {'sex_upon_outcome': 'Intact Male'},
            {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
        ]}

    # Retrieve data from MongoDB
    data = shelter.read_all(query)
    df = pd.DataFrame.from_records(data)
    df.drop(columns=['_id'], inplace=True)

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    
    return data, columns

# Update Histogram based on DataTable
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available.")
    
    dff = pd.DataFrame.from_dict(viewData)
    figure = px.histogram(dff, x='breed', template="plotly", color_discrete_sequence=px.colors.qualitative.Plotly)
    figure.update_layout(
        font=dict(family="Arial, sans-serif", size=14, color="black"),
        paper_bgcolor="white",
        plot_bgcolor="white",
        title_text="Animal Breed Distribution",
        xaxis_title="Breed",
        yaxis_title="Count",
    )
    return dcc.Graph(figure=figure)

# Update Map based on DataTable selection
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available.")
    
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    return dl.Map(style={'width': '1000px', 'height': '500px'},
                  center=[30.75, -97.48], zoom=10, children=[
                      dl.TileLayer(id="base-layer-id"),
                      dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                                children=[
                                    dl.Tooltip(dff.iloc[row, 4]),
                                    dl.Popup([
                                        html.H1("Animal Name"),
                                        html.P(dff.iloc[row, 9])
                                    ])
                                ])
                  ])

app.run_server(debug=True)

Dash app running on http://127.0.0.1:31103/
